In [1]:
import pandas as pd

D:\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
df = pd.read_csv('data.csv')

In [3]:
df = df[['reviews.text', 'reviews.rating']]

In [4]:
df['reviews.rating'].replace({1:0,2:0,3:0,4:1,5:1}, inplace=True)

In [5]:
#Removing stopwords

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [7]:
stop_words = set(stopwords.words('english'))

In [8]:
def preprocess(text):
    word_tokens = word_tokenize(text)
    processed = [w for w in word_tokens if not w.lower() in stop_words]
    return ' '.join(processed)

In [9]:
df['filtered'] = df['reviews.text'].map(preprocess)

In [10]:
df['filtered']

0       thought would big small paper turn like palm ....
1            kindle light easy use especially beach ! ! !
2       Didnt know much 'd use kindle went lower end ....
3       100 happy purchase . caught sale really good p...
4       Solid entry level Kindle . Great kids . Gifted...
                              ...                        
4995                 great tablet price . Amazon good job
4996    tablet perfect size easy use . Read , play gam...
4997    Purchased son . room upgrade memory allow book...
4998    thoughts getting 5 year old , get screen prote...
4999                 steal , 8 gb model well.This punch..
Name: filtered, Length: 5000, dtype: object

In [11]:
#Using simple feed forward neural network

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
cv = CountVectorizer()

In [14]:
text = cv.fit_transform(df['filtered']).toarray()

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(text,df['reviews.rating'], shuffle=True, test_size=0.2, random_state=8)

In [17]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [18]:
model = Sequential(
[
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
]
    
)

In [19]:
model.compile(optimizer='adam', metrics='accuracy', loss='binary_crossentropy')

In [20]:
model.fit(X_train, Y_train, epochs=10, validation_data=(X_test,Y_test))

Epoch 1/10
125/125 [==============================] - 1s 6ms/step - loss: 0.3393 - accuracy: 0.9293 - val_loss: 0.2424 - val_accuracy: 0.9390
Epoch 2/10
125/125 [==============================] - 1s 4ms/step - loss: 0.1697 - accuracy: 0.9367 - val_loss: 0.1954 - val_accuracy: 0.9430
Epoch 3/10
125/125 [==============================] - 1s 4ms/step - loss: 0.1097 - accuracy: 0.9555 - val_loss: 0.1892 - val_accuracy: 0.9540
Epoch 4/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0649 - accuracy: 0.9803 - val_loss: 0.2337 - val_accuracy: 0.9520
Epoch 5/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0358 - accuracy: 0.9908 - val_loss: 0.2577 - val_accuracy: 0.9470
Epoch 6/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0213 - accuracy: 0.9955 - val_loss: 0.2974 - val_accuracy: 0.9460
Epoch 7/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0134 - accuracy: 0.9975 - val_loss: 0.3283 - val_accuracy: 0.9490
Epoch 

In [21]:
#Using LSTM

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [23]:
tokenizer=Tokenizer(num_words=500)

In [24]:
tokenizer.fit_on_texts(df['filtered'])

In [25]:
text = tokenizer.texts_to_sequences(df['filtered'])

In [26]:
vocab_size = len(tokenizer.word_index) + 1

In [27]:
from keras.preprocessing.sequence import pad_sequences
maxlen=100
text = pad_sequences(text, padding='post', maxlen=maxlen)

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(text,df['reviews.rating'], shuffle=True, test_size=0.2, random_state=8)

In [29]:
from tensorflow.keras.layers import LSTM,Embedding, Flatten
embedding_dim = 50
model = Sequential(
[
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen),
    LSTM(8, activation='relu'),
    Flatten(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
]
    
)
model.compile(optimizer='adam', metrics='accuracy', loss='binary_crossentropy')

In [30]:
model.fit(X_train, Y_train, epochs=10, validation_data=(X_test,Y_test))

Epoch 1/10
125/125 [==============================] - 6s 38ms/step - loss: 0.3790 - accuracy: 0.9367 - val_loss: 0.2314 - val_accuracy: 0.9390
Epoch 2/10
125/125 [==============================] - 4s 35ms/step - loss: 0.2376 - accuracy: 0.9367 - val_loss: 0.2304 - val_accuracy: 0.9390
Epoch 3/10
125/125 [==============================] - 4s 34ms/step - loss: 0.2370 - accuracy: 0.9367 - val_loss: 0.2302 - val_accuracy: 0.9390
Epoch 4/10
125/125 [==============================] - 4s 34ms/step - loss: 0.2370 - accuracy: 0.9367 - val_loss: 0.2329 - val_accuracy: 0.9390
Epoch 5/10
125/125 [==============================] - 4s 35ms/step - loss: 0.2376 - accuracy: 0.9367 - val_loss: 0.2300 - val_accuracy: 0.9390
Epoch 6/10
125/125 [==============================] - 4s 34ms/step - loss: 0.2356 - accuracy: 0.9367 - val_loss: 0.5783 - val_accuracy: 0.9390
Epoch 7/10
125/125 [==============================] - 4s 34ms/step - loss: 0.2558 - accuracy: 0.9367 - val_loss: 0.2299 - val_accuracy: 0.9390

In [31]:
#Using CNN

In [32]:
from tensorflow.keras.layers import Conv1D
embedding_dim = 50
model = Sequential(
[
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen),
    Conv1D(8, 5, activation='relu'),
    Flatten(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
]
    
)
model.compile(optimizer='adam', metrics='accuracy', loss='binary_crossentropy')

In [33]:
model.fit(X_train, Y_train, epochs=10, validation_data=(X_test,Y_test))

Epoch 1/10
125/125 [==============================] - 2s 13ms/step - loss: 0.2676 - accuracy: 0.9298 - val_loss: 0.2331 - val_accuracy: 0.9390
Epoch 2/10
125/125 [==============================] - 2s 12ms/step - loss: 0.2044 - accuracy: 0.9367 - val_loss: 0.1840 - val_accuracy: 0.9390
Epoch 3/10
125/125 [==============================] - 2s 12ms/step - loss: 0.1475 - accuracy: 0.9367 - val_loss: 0.1927 - val_accuracy: 0.9390
Epoch 4/10
125/125 [==============================] - 2s 12ms/step - loss: 0.1183 - accuracy: 0.9448 - val_loss: 0.1809 - val_accuracy: 0.9450
Epoch 5/10
125/125 [==============================] - 2s 12ms/step - loss: 0.0945 - accuracy: 0.9588 - val_loss: 0.1961 - val_accuracy: 0.9430
Epoch 6/10
125/125 [==============================] - 2s 12ms/step - loss: 0.0678 - accuracy: 0.9750 - val_loss: 0.2149 - val_accuracy: 0.9440
Epoch 7/10
125/125 [==============================] - 2s 12ms/step - loss: 0.0423 - accuracy: 0.9883 - val_loss: 0.2766 - val_accuracy: 0.9460